**UNZIPPEN

Problemen met lengte van de paden in de sentinel datastructuur
Unzippen lukt wel indien niet te ver "verstopt" in de windows tree...

In [1]:
import os
from glob import glob
import zipfile
import shutil

In [2]:
input_zip_dir = "C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/"

In [3]:
def extract(zip_filepath, extraction_dir):
    z = zipfile.ZipFile(zip_filepath)
    z.extractall(extraction_dir)

In [4]:
extraction_zip_dir = "C:/SS/UZ/"

In [6]:
for zip_file in glob(os.path.join(input_zip_dir, "**", "*.zip"), recursive=True):
    extract(zip_file, extraction_zip_dir)

groeperen op datum

In [5]:
if not os.path.isdir("C:/SS/DT/"):
    os.mkdir("C:/SS/DT/")

In [12]:
for filename in os.listdir(extraction_zip_dir):
    date = filename[-20:-12]
    date_folders = "C:/SS/DT/" + date
    if not os.path.isdir(date_folders):
        os.mkdir(date_folders)

Opnieuw probleem van te lange paden?

In [ ]:
# moven lukt niet, te lange paden?
for filename in os.listdir(extraction_zip_dir):
    date = filename[-20:-12]
    shutil.move(src = filename, dst = "C:/SS/DT/" + date)

Deleten van wat we niet nodig hebben.

Filteren van de granules en enkel deze behouden die (deels) binnen de bounding box van Vlaanderen vallen....µ

Voor de data van juli 2016, +/- 30Gb overbodige data opgekuist op deze manier...

In [6]:
# granules die binnen de bounding box van Vlaanderen vallen...
selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
selected_folders =[]

In [7]:
for folder in glob(os.path.join("C:/SS/DT/", "**", "*/GRANULE/*"), recursive=True):
    for granule in selected_granules:
        if granule in folder:
            selected_folders.append(folder)

In [8]:
for folder in glob(os.path.join("C:/SS/DT/", "**", "*/GRANULE/*"), recursive=True):
    if folder not in selected_folders:
        shutil.rmtree(folder)

** jp2 --> Gtiff

In [9]:
import rasterio
import rsgislib
from rsgislib import imageutils, RSGISPyUtils
import osgeo
from osgeo import gdal, osr, ogr
import subprocess

In [10]:
# Enable GDAL/OGR exceptions
gdal.UseExceptions()
osgeo.gdal.SetConfigOption("GDAL_DATA", "C:/Users/jeroen.dereu@inbo.be/Software/gdal-2.1.3/data")

In [11]:
input_jp2_dir = "C:/SS/DT/"

In [12]:
#listdir checken
folders = []
for folder in os.listdir(input_jp2_dir): 
    folders.append("C:/SS/DT/"+folder+"/")

In [13]:
for folder in folders:
    print(folder)

C:/SS/DT/20160703/
C:/SS/DT/20160707/
C:/SS/DT/20160720/
C:/SS/DT/20160723/


In [15]:
# mosaic individual bands --- WORKING!
for folder in folders:
    selected_granules = ["T31UDS", "T31UES", "T31UFS", "T31UGS", "T31UDT", "T31UET", "T31UFT", "T31UGT"]
    selected_images = []
    B01 = []
    B02 = []
    B03 = []
    B04 = []
    B05 = []
    B06 = []
    B07 = []
    B08 = []
    B8A = []
    B09 = []
    B10 = []
    B11 = []
    B12 = []
    overview = []
    for path in glob(os.path.join(folder, "**", "*.jp2"), recursive=True):
        for granule in selected_granules:
            if granule in path:
                selected_images.append(path)
    for image in selected_images:
        if image.endswith("B01.jp2"):
            B01.append(image)
        elif image.endswith("B02.jp2"):
            B02.append(image)
        elif image.endswith("B03.jp2"):
            B03.append(image)
        elif image.endswith("B04.jp2"):
            B04.append(image)
        elif image.endswith("B05.jp2"):
            B05.append(image)
        elif image.endswith("B06.jp2"):
            B06.append(image)
        elif image.endswith("B07.jp2"):
            B07.append(image)
        elif image.endswith("B08.jp2"):
            B08.append(image)        
        elif image.endswith("B8A.jp2"):
            B8A.append(image)
        elif image.endswith("B09.jp2"):
            B09.append(image)
        elif image.endswith("B10.jp2"):
            B10.append(image)        
        elif image.endswith("B11.jp2"):
            B11.append(image)      
        elif image.endswith("B12.jp2"):
            B12.append(image)         
        else:
            overview.append(image)
    if not os.path.isdir(folder + "/merged_bands"):
        os.mkdir(folder + "/merged_bands")
    output_folder = folder + "/merged_bands/"
    output_B01 = output_folder + "B01" + ".tif"
    output_B02 = output_folder + "B02" + ".tif"
    output_B03 = output_folder + "B03" + ".tif"
    output_B04 = output_folder + "B04" + ".tif"
    output_B05 = output_folder + "B05" + ".tif"
    output_B06 = output_folder + "B06" + ".tif"
    output_B07 = output_folder + "B07" + ".tif"
    output_B08 = output_folder + "B08" + ".tif"
    output_B8A = output_folder + "B8A" + ".tif"
    output_B09 = output_folder + "B09" + ".tif"   
    output_B10 = output_folder + "B10" + ".tif"
    output_B11 = output_folder + "B11" + ".tif"
    output_B12 = output_folder + "B12" + ".tif"
    output_overview = output_folder + "overview" + ".tif"
    gdal.Warp(destNameOrDestDS = output_B01, 
              srcDSOrSrcDSTab = B01, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B02, 
              srcDSOrSrcDSTab = B02, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B03, 
              srcDSOrSrcDSTab = B03, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")        
    gdal.Warp(destNameOrDestDS = output_B04, 
              srcDSOrSrcDSTab = B04, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")           
    gdal.Warp(destNameOrDestDS = output_B05, 
              srcDSOrSrcDSTab = B05, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")             
    gdal.Warp(destNameOrDestDS = output_B06, 
              srcDSOrSrcDSTab = B06, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B07, 
              srcDSOrSrcDSTab = B07, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B08, 
              srcDSOrSrcDSTab = B08, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B8A, 
              srcDSOrSrcDSTab = B8A, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B09, 
              srcDSOrSrcDSTab = B09, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B10, 
              srcDSOrSrcDSTab = B10, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B11, 
              srcDSOrSrcDSTab = B11, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_B12, 
              srcDSOrSrcDSTab = B12, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")
    gdal.Warp(destNameOrDestDS = output_overview, 
              srcDSOrSrcDSTab = overview, 
              dstSRS = "EPSG:4326", 
              srcSRS = "EPSG:32631",
              cropToCutline = True,
              cutlineDSName = "C:/WERKMAP/Anaconda/sentinelsat/geojson/footprint_flanders_geojson.geojson")

In [17]:
# converten van jp2 naar Geotiff, voor elk beeld apart --- WORKING!
for folder in folders:
    for path in glob(os.path.join(folder, "**", "*.jp2"), recursive=True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        if not os.path.isdir(folder + "/ind_tifs"):
            os.mkdir(folder + "/ind_tifs")
        output_folder = folder + "/ind_tifs/"
        output_file = output_folder + filename + "tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  dstSRS = "EPSG:4326", 
                  srcSRS = "EPSG:32631")

In [43]:
# stacked bands vb. RGB --- WORKING!
for folder in folders:
    RGB = []
    for path in glob(os.path.join(folder, "**", "Merge", "*tif"), recursive=True):
        if path.endswith("B02.tif"):
            RGB.append(path)
        if path.endswith("B03.tif"):
            RGB.append(path) 
        if path.endswith("B04.tif"):
            RGB.append(path)
    bandNamesList = ["Blue", "Green", "Red"]
    if not os.path.isdir(folder + "/stacked/"):
        os.mkdir(folder + "/stacked/")
    outputImage = folder + "/stacked/" + "RGB.tif"
    gdalFormat = "GTiff"
    dataType = rsgislib.TYPE_16UINT
    imageutils.stackImageBands(RGB, bandNamesList, outputImage, None, 0, gdalFormat, dataType)